# ML Zoomcamp Capstone Project II - New Exporter Success 

## Data Preparation

### Export Records Cleanup

In [1]:
#Libraries
import numpy as np
import pandas as pd

In [2]:
#Define data types:
data_types = {"nit":str, "raz_sial":str, "dpto1":str, "posar":str, "cod_pai4":str, "fech":str, "modad":str, 
              "pbk":float, "pnk":float, "fobdol":float, "fobpes":float, "agrena":float, "fletes":float, 
              "seguro":float, "otrosg":float}

In [3]:
#Read database
exports = pd.read_csv("./ExportRecords.csv", usecols=list(data_types.keys()), dtype=data_types)

In [4]:
#Explore variables
exports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3228341 entries, 0 to 3228340
Data columns (total 15 columns):
 #   Column    Dtype  
---  ------    -----  
 0   nit       object 
 1   raz_sial  object 
 2   dpto1     object 
 3   posar     object 
 4   cod_pai4  object 
 5   fech      object 
 6   modad     object 
 7   pbk       float64
 8   pnk       float64
 9   fobdol    float64
 10  fobpes    float64
 11  agrena    float64
 12  fletes    float64
 13  seguro    float64
 14  otrosg    float64
dtypes: float64(8), object(7)
memory usage: 369.5+ MB


In [5]:
#Create date variables
exports["year"] = exports["fech"].str[0:2]
exports["month"] = exports["fech"].str[2:4]

In [6]:
#Export types
exports["modad"].value_counts()

198    2716670
199     317560
104      78696
2        75016
107      15202
4        10371
401       5140
201       5038
402       3056
310       1465
403        125
600          2
Name: modad, dtype: int64

In [7]:
#Exclude all records of non-commercial exports
commercial_exports = ["198"]
exports = exports[exports["modad"].isin(commercial_exports)]

In [8]:
#Exclude exports destined to local free trade zones
exports = exports[exports["cod_pai4"]!="XCF"]

In [9]:
#Exclude export records where country of destination is not specified
exports = exports[exports["cod_pai4"]!="ZZZ"]

In [10]:
#Exclude one time exporters
one_timers = (exports["nit"].value_counts()==1)*1
one_timers = list(one_timers[one_timers==1].index)

def tag_one_timers(nit):
    if nit in one_timers:
        return 1
    else:
        return 0
    
exports["one_timers"] = exports["nit"].apply(tag_one_timers)
exports = exports[exports["one_timers"]==0]

In [11]:
#Total export records after filtering data
print(f"Total export records after filtering data: {exports.shape}")

Total export records after filtering data: (2704478, 18)


### Exporter location

In [12]:
#Determine exporter headquarters
exporter_location = exports[["nit", "dpto1"]].value_counts()
exporter_location = exporter_location.reset_index()
exporter_location.rename({0:"quantity"}, axis=1, inplace=True)
exporter_location.sort_values(by=["nit", "quantity"], ascending=[True, False], inplace=True)
exporter_location.drop_duplicates(subset=["nit"], keep="first", inplace=True, ignore_index=True)
exporter_location.head(2)

,nit,dpto1,quantity
0,000002988341,11,2
1,000006716172,11,5


### Aggregate Exports

In [13]:
#Aggregate by year, export, product and country of destination
exports_agg = exports.groupby(by=["nit", "posar", "cod_pai4", "year"]).sum()
exports_agg = exports_agg[["pbk", "pnk", "fobdol", "fobpes", "agrena", "fletes", "seguro", "otrosg"]]
exports_agg.reset_index(inplace=True)

In [14]:
#Change type of year variable
exports_agg["year"] = exports_agg["year"].apply(int)

In [15]:
#Eliminate exports valued in less than USD 1000 during the year
exports_agg = exports_agg[exports_agg["fobdol"]>1000]
exports_agg.reset_index(inplace=True, drop=True)

In [16]:
#Sort values and reset index
exports_agg.reset_index(inplace=True, drop=True)
exports_agg.sort_values(by=["nit","cod_pai4", "posar", "year"], inplace=True)

In [17]:
#Total aggregate export records after filtering the data
print(f"Total aggregate export records after filtering data: {exports_agg.shape}")

Total aggregate export records after filtering data: (419718, 12)


### Export experience

In [18]:
#Number of markets in which the exporter actively trades 
active_markets = exports_agg.groupby(by=["nit", "year"])[["cod_pai4"]].nunique()
active_markets = active_markets.reset_index()
active_markets.rename({"cod_pai4":"active_markets"}, axis=1, inplace=True)
active_markets.head(2)

,nit,year,active_markets
0,000002988341,19,2
1,000006716172,19,2


In [19]:
#Number of products in that the exporter already trades 
active_products = exports_agg.groupby(by=["nit", "year"])["posar"].nunique()
active_products = active_products.reset_index()
active_products.rename({"posar":"active_products"}, axis=1, inplace=True)
active_products.head(2)

,nit,year,active_products
0,000002988341,19,1
1,000006716172,19,1


In [20]:
#Value of annual exports
exports_value = exports_agg.groupby(by=["nit", "year"])["fobdol"].sum()
exports_value = exports_value.reset_index()
exports_value.rename({"fobdol":"exports_value"}, axis=1, inplace=True)
exports_value.head(2)

,nit,year,exports_value
0,000002988341,19,6591.90
1,000006716172,19,6311.93


In [21]:
#Exporting experience in years
exporter_years = exports_agg.groupby(by=["nit", "year"]).count()
exporter_years = exporter_years.reset_index()

#DF to capture years 
active_years = pd.DataFrame(columns=["nit", "year", "active_years"])

#Obtain exporter aggregate data
for exporter in exporter_years["nit"].unique():
    records = exporter_years[exporter_years["nit"]==exporter]["year"]
    for year in records.unique():        
        exporter_row = pd.DataFrame(data=[[exporter, year, sum(records.values<year)]],
                                    columns=["nit", "year", "active_years"])
        active_years = pd.concat([active_years, exporter_row], ignore_index=True)

active_years.head()

,nit,year,active_years
0,000002988341,19,0
1,000006716172,19,0
2,000010943378,19,0
3,000019473179,19,0
4,000037838591,21,0


In [22]:
#Merge DFs to obtain dataframe of exporter experience
exporter_agg = active_markets.merge(active_products, on=["nit", "year"], how="outer")
exporter_agg.reset_index(drop=True, inplace=True)
exporter_agg = exporter_agg.merge(exports_value, on=["nit", "year"], how="outer")
exporter_agg.reset_index(drop=True, inplace=True)
exporter_agg = exporter_agg.merge(active_years, on=["nit", "year"], how="outer")
exporter_agg.reset_index(drop=True, inplace=True)
exporter_agg.head()

,nit,year,active_markets,active_products,exports_value,active_years
0,000002988341,19,2,1,6591.90,0
1,000006716172,19,2,1,6311.93,0
2,000010943378,19,2,1,7510.03,0
3,000019473179,19,7,1,49384.81,0
4,000037838591,21,1,6,14347.90,0


### New Exporters

In [23]:
#Times exporter has traded specific product to particular market
export_combination = exports_agg.groupby(by=["nit", "cod_pai4", "posar"])["year"].count()
export_combination = export_combination.reset_index()
export_combination.rename({"year":"duration"}, axis=1, inplace=True)
export_combination.head(2)

,nit,cod_pai4,posar,duration
0,000002988341,DEU,301110090,1
1,000002988341,NOR,301110090,1


In [24]:
#Merge with aggregate exporter data_base and tag new exporters in years of entering new market with new products
new_exporters = exports_agg[["nit", "posar", "cod_pai4", "year"]].merge(export_combination, 
                                                                        on=["nit", "cod_pai4", "posar"], how="left")
new_exporters.head(2)

,nit,posar,cod_pai4,year,duration
0,000002988341,301110090,DEU,19,1
1,000002988341,301110090,NOR,19,1


In [25]:
#Drop records from 2016 as it is now possible to determine new exporters this year without data from 2015
new_exporters = new_exporters[new_exporters["year"]!=16]
new_exporters.reset_index(drop=True, inplace=True)

In [26]:
#Drop records from 2022 as it is now possible to determine success of new exporters this year without data from 2023
new_exporters = new_exporters[new_exporters["year"]!=22]
new_exporters.reset_index(drop=True, inplace=True)

In [27]:
#Drop records were duration is more than 7 years as these can't be new product-market combinations for an exporter
new_exporters = new_exporters[new_exporters["duration"]<7]
new_exporters.reset_index(drop=True, inplace=True)

In [28]:
#Sort new_exporters by exporter, product, market combination and year
new_exporters.sort_values(by=["nit", "posar", "cod_pai4", "year"], inplace=True)

In [29]:
#Leave only first appearance of the exporter, product, market combination
new_exporters.drop_duplicates(subset=["nit", "posar", "cod_pai4"], keep="first", inplace=True, ignore_index=True)

In [30]:
#Number of new exporters
print(f"Number of new exporters:", new_exporters.shape)

Number of new exporters: (135639, 5)


In [31]:
#Determine successful new exporters
def tag_success(duration):
    if duration>1:
        return 1
    else: 
        return 0
    

new_exporters["success"] = new_exporters["duration"].apply(tag_success)

In [32]:
#Explore database with data for new exporters
new_exporters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135639 entries, 0 to 135638
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   nit       135639 non-null  object
 1   posar     135639 non-null  object
 2   cod_pai4  135639 non-null  object
 3   year      135639 non-null  int64 
 4   duration  135639 non-null  int64 
 5   success   135639 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 6.2+ MB


### Merge Dataframes

In [33]:
#Translate key column names for merge
new_exporters.rename({"cod_pai4":"market", "posar":"product"}, axis=1, inplace=True)
exports_agg.rename({"cod_pai4":"market", "posar":"product"}, axis=1, inplace=True)

In [34]:
#Add info on new exporters to aggregate exports
exports_new = new_exporters.merge(exports_agg, how="left", on=["nit", "year", "product", "market"])
exports_new.reset_index(inplace=True, drop=True)

In [35]:
#Append aggregate information on export volumes, product and market portfolio
exports_new = exports_new.assign(pre_year=exports_new["year"]-1)
exports_new = exports_new.merge(exporter_agg, how="left", left_on=["nit", "pre_year"], 
                                          right_on=["nit", "year"])
exports_new.reset_index(inplace=True, drop=True)

In [36]:
#Append export location 
exports_new = exports_new.merge(exporter_location, how="left", on=["nit"])
exports_new.reset_index(inplace=True, drop=True)

In [37]:
#Explore resulting database
exports_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135639 entries, 0 to 135638
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   nit              135639 non-null  object 
 1   product          135639 non-null  object 
 2   market           135639 non-null  object 
 3   year_x           135639 non-null  int64  
 4   duration         135639 non-null  int64  
 5   success          135639 non-null  int64  
 6   pbk              135639 non-null  float64
 7   pnk              135639 non-null  float64
 8   fobdol           135639 non-null  float64
 9   fobpes           135639 non-null  float64
 10  agrena           135639 non-null  float64
 11  fletes           135639 non-null  float64
 12  seguro           135639 non-null  float64
 13  otrosg           135639 non-null  float64
 14  pre_year         135639 non-null  object 
 15  year_y           111479 non-null  object 
 16  active_markets   111479 non-null  floa

In [38]:
#Fill in na active_markets, active_products, exports_value and active_years
exports_new_final = exports_new.fillna(value=0)

In [39]:
#Rename and drop year variables
exports_new_final.rename({"year_x":"year"}, axis=1, inplace=True)
exports_new_final.drop(labels=["year_y", "pre_year"], axis=1, inplace=True)

In [40]:
exports_new_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135639 entries, 0 to 135638
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   nit              135639 non-null  object 
 1   product          135639 non-null  object 
 2   market           135639 non-null  object 
 3   year             135639 non-null  int64  
 4   duration         135639 non-null  int64  
 5   success          135639 non-null  int64  
 6   pbk              135639 non-null  float64
 7   pnk              135639 non-null  float64
 8   fobdol           135639 non-null  float64
 9   fobpes           135639 non-null  float64
 10  agrena           135639 non-null  float64
 11  fletes           135639 non-null  float64
 12  seguro           135639 non-null  float64
 13  otrosg           135639 non-null  float64
 14  active_markets   135639 non-null  float64
 15  active_products  135639 non-null  float64
 16  exports_value    135639 non-null  floa

## Additional Features

### Append additional information to create extra features

In [41]:
#Size of new market
gdp_pc = pd.read_excel("./weo_gdp.xlsx", sheet_name="Data")
gdp_pc.rename({"Market":"market", "Country":"country"}, axis=1, inplace=True)

gdp_pc = gdp_pc.melt(id_vars=["market", "country"], var_name="year_num", value_name="gdp_pc")

gdp_pc["year"] = gdp_pc["year_num"]-2000

gdp_pc.head(2)

,market,country,year_num,gdp_pc,year
0,AFG,Afghanistan,2016,2492.65,16
1,ALB,Albania,2016,12597.30,16


In [42]:
#Identify new markets where FTA is available

#Multilateral FTA
can = ["BOL", "ECU", "PER"]
caricom = ["ATG", "BHS", "BRB", "BLZ", "DMA", "GRD", "GUY", "HTI", "JAM", "MSR", "KNA", "LCA", "VCT", "SUR", "TTO"]
efta = ["ISL", "LIE", "NOR", "CHE"]
eu = ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", 
      "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]
mercosur = ["ARG", "BRA", "PRY", "URY"]
pa = ["CHL", "MEX", "PER"]
tri_cent_america = ["GTM", "HND", "SLV"]

#Bilateral fta
bilateral = ["CAN", "CHL","CRI", "CUB", "GBR", "TSR", "KOR", "MEX", "PAN", "USA", "VEN"]

#All FTAs
fta_all = can + caricom + efta + mercosur + eu + pa + tri_cent_america + bilateral

def fta_identifier(market):
    if market in fta_all:
        return 1
    else:
        return 0

exports_new_final["fta"] = exports_new_final["market"].apply(fta_identifier)

In [43]:
#Continent and languages of new markets
country_data = pd.read_csv("./CountryData.csv")
country_data.drop(["name", "Unnamed: 0"], axis=1, inplace=True)
country_data.head(2)

,market,region,subregion,languages_dict,languages
0,AFG,Asia,Southern Asia,"[{'iso639_1': 'ps', 'iso639_2': 'pus', 'name':...",Pashto;Uzbek;Turkmen
1,ALA,Europe,Northern Europe,"[{'iso639_1': 'sv', 'iso639_2': 'swe', 'name':...",Swedish


In [44]:
#Merge datasets
exports_new_final = exports_new_final.merge(gdp_pc[["market", "year", "gdp_pc"]], how="left", on=["market", "year"])
exports_new_final.reset_index(inplace=True, drop=True)
exports_new_final = exports_new_final.merge(country_data, how="left", on="market")
exports_new_final.reset_index(inplace=True, drop=True)

In [45]:
#Identify markets where Spanish is an official languae
def tag_spanish(languages):
    languages_list = languages.split(";")
    if "Spanish" in languages_list:
        return 1
    else:
        return 0
    
exports_new_final["spanish"] = exports_new_final["languages"].apply(tag_spanish) 

In [46]:
#Identify markets where English is an official languae
def tag_english(languages):
    languages_list = languages.split(";")
    if "English" in languages_list:
        return 1
    else:
        return 0
    
exports_new_final["english"] = exports_new_final["languages"].apply(tag_english) 

In [47]:
exports_new_final.head()

,nit,product,market,year,duration,success,pbk,pnk,fobdol,fobpes,...,dpto1,quantity,fta,gdp_pc,region,subregion,languages_dict,languages,spanish,english
0,000002988341,301110090,DEU,19,1,0,403.00,322.40,4495.40,15413063.30,...,11,2,1,54181.28,Europe,Central Europe,"[{'iso639_1': 'de', 'iso639_2': 'deu', 'name':...",German,0,0
1,000002988341,301110090,NOR,19,1,0,174.00,139.20,2096.50,7188122.80,...,11,2,1,63164.04,Europe,Northern Europe,"[{'iso639_1': 'no', 'iso639_2': 'nor', 'name':...",Norwegian;Norwegian Bokmål;Norwegian Nynorsk,0,0
2,000006716172,301110090,CAN,19,1,0,98.11,78.49,1716.93,5889550.64,...,11,5,1,49560.84,Americas,Northern America,"[{'iso639_1': 'en', 'iso639_2': 'eng', 'name':...",English;French,0,1
3,000006716172,301110090,JPN,19,1,0,322.00,257.60,4595.00,15736064.80,...,11,5,0,41699.18,Asia,Eastern Asia,"[{'iso639_1': 'ja', 'iso639_2': 'jpn', 'name':...",Japanese,0,0
4,000010943378,301110090,GBR,19,1,0,186.00,148.80,4126.00,14037972.32,...,11,5,1,47567.99,Europe,Northern Europe,"[{'iso639_1': 'en', 'iso639_2': 'eng', 'name':...",English,0,1


## Final Dataset

In [48]:
exports_new_final.columns

Index(['nit', 'product', 'market', 'year', 'duration', 'success', 'pbk', 'pnk',
       'fobdol', 'fobpes', 'agrena', 'fletes', 'seguro', 'otrosg',
       'active_markets', 'active_products', 'exports_value', 'active_years',
       'dpto1', 'quantity', 'fta', 'gdp_pc', 'region', 'subregion',
       'languages_dict', 'languages', 'spanish', 'english'],
      dtype='object')

In [49]:
exports_new_final.head(2)

,nit,product,market,year,duration,success,pbk,pnk,fobdol,fobpes,...,dpto1,quantity,fta,gdp_pc,region,subregion,languages_dict,languages,spanish,english
0,000002988341,301110090,DEU,19,1,0,403.0,322.4,4495.4,15413063.3,...,11,2,1,54181.28,Europe,Central Europe,"[{'iso639_1': 'de', 'iso639_2': 'deu', 'name':...",German,0,0
1,000002988341,301110090,NOR,19,1,0,174.0,139.2,2096.5,7188122.8,...,11,2,1,63164.04,Europe,Northern Europe,"[{'iso639_1': 'no', 'iso639_2': 'nor', 'name':...",Norwegian;Norwegian Bokmål;Norwegian Nynorsk,0,0


In [50]:
#Translate var names
exports_new_final.rename({"nit":"exporter", "fletes":"freight", "seguro":"insurance", "otrosg":"other_expenses",
                         "agrena":"product_national"}, axis=1, inplace=True) 

In [51]:
#Export final database
exports_new_final.to_csv("./NewExportersExtended.csv", sep=";", index=False)